In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from numpy import mean, std, sqrt
import datetime
from scipy.stats import ttest_ind
import scipy.stats as stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from lifelines import KaplanMeierFitter, CoxPHFitter

In [ ]:
# Enter Parameters

# read_from_filepath = r"ENTER FILE PATH TO READ DATA FROM"

In [ ]:
result_df = pd.read_csv(read_from_filepath)
result_df.head()

,Date,Day of Week,Time,Before 1 PM,entry_time,contract_symbol,moneyness,Option Type,Strike Distance,DTE,...,5%,10%,15%,20%,25%,30%,35%,40%,45%,50%
0,2024-01-18,Thursday,11:38:00,Y,09:31:00,SPY240118C00471000,ITM,Call,-4,0,...,True,True,True,True,True,True,True,True,False,False
1,2024-01-18,Thursday,11:36:00,Y,09:31:00,SPY240118C00472000,ITM,Call,-3,0,...,True,True,True,True,True,True,True,True,True,True
2,2024-01-18,Thursday,11:36:00,Y,09:31:00,SPY240118C00473000,ITM,Call,-2,0,...,True,True,True,True,True,True,True,True,True,True
3,2024-01-18,Thursday,11:36:00,Y,09:31:00,SPY240118C00474000,ITM,Call,-1,0,...,True,True,True,True,True,True,True,True,True,True
4,2024-01-18,Thursday,11:36:00,Y,09:31:00,SPY240118C00475000,OTM,Call,0,0,...,True,True,True,True,True,True,True,True,True,True


### Descriptive Statistics

In [4]:
# Calculate descriptive statistics for each profit target level
profit_targets = ['5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%', '45%', '50%']
profit_summary = result_df[profit_targets].apply(lambda x: x.mean()) * 100  # Convert to percentage
print("Percentage of entries achieving each profit target level:")
print(profit_summary)

Percentage of entries achieving each profit target level:
5%     83.545850
10%    75.173799
15%    68.331412
20%    62.532391
25%    57.666008
30%    53.414915
35%    49.822491
40%    46.936679
45%    44.254061
50%    41.915047
dtype: float64


### Distribution Analysis by Factors

In [5]:
# Group by relevant factors and calculate success rates
grouped_summary = result_df.groupby(['Day of Week', 'Option Type', 'moneyness', 'Strike Distance', 'DTE'])[profit_targets].mean() * 100
print("Grouped summary by day of week, option type, moneyness, strike distance, and DTE:")
print(grouped_summary)

Grouped summary by day of week, option type, moneyness, strike distance, and DTE:
                                                               5%         10%  \
Day of Week Option Type moneyness Strike Distance DTE                           
Friday      Call        ATM        0              0    100.000000  100.000000   
                                                  3    100.000000  100.000000   
                        ITM       -9              0      0.000000    0.000000   
                                                  3      0.000000    0.000000   
                                  -8              0     33.333333   33.333333   
...                                                           ...         ...   
Wednesday   Put         OTM       -2              1     87.865169   80.449438   
                                                  2     34.782609   26.086957   
                                  -1              0     80.487805   73.614191   
                           

### Statistical Testing: T-Tests

In [6]:

# Convert boolean values in profit target columns to integers (1 for True, 0 for False)
for target in profit_targets:
    result_df[target] = result_df[target].astype(int)

# Separate data into two groups: before 1 PM and after 1 PM
before_1pm = result_df[result_df['Before 1 PM'] == 'Y']
after_1pm = result_df[result_df['Before 1 PM'] == 'N']

# Perform T-test for each profit target
for target in profit_targets:
    t_stat, p_val = ttest_ind(before_1pm[target], after_1pm[target], nan_policy='omit')
    print(f"{target}: T-statistic = {t_stat:.2f}, P-value = {p_val:.4f}")


5%: T-statistic = 25.62, P-value = 0.0000
10%: T-statistic = 30.77, P-value = 0.0000
15%: T-statistic = 34.63, P-value = 0.0000
20%: T-statistic = 36.47, P-value = 0.0000
25%: T-statistic = 38.28, P-value = 0.0000
30%: T-statistic = 40.13, P-value = 0.0000
35%: T-statistic = 42.31, P-value = 0.0000
40%: T-statistic = 42.91, P-value = 0.0000
45%: T-statistic = 43.38, P-value = 0.0000
50%: T-statistic = 43.28, P-value = 0.0000


### Confidence Intervals for Each Target Level

In [7]:
# Calculate 95% confidence intervals for each target
confidence_intervals = {}
for target in profit_targets:
    successes = result_df[target].sum()
    total = len(result_df)
    proportion = successes / total
    ci_low, ci_high = stats.binom.interval(0.95, total, proportion)
    confidence_intervals[target] = (ci_low / total * 100, ci_high / total * 100)

print("95% Confidence Intervals for each profit target level:")
for target, ci in confidence_intervals.items():
    print(f"{target}: {ci[0]:.2f}% - {ci[1]:.2f}%")

95% Confidence Intervals for each profit target level:
5%: 83.30% - 83.79%
10%: 74.89% - 75.46%
15%: 68.02% - 68.64%
20%: 62.21% - 62.85%
25%: 57.34% - 57.99%
30%: 53.09% - 53.74%
35%: 49.49% - 50.15%
40%: 46.61% - 47.27%
45%: 43.93% - 44.58%
50%: 41.59% - 42.24%


### Sensitivity Analysis of Composite Scores

In [8]:
# Define varying weights for sensitivity analysis
weights_list = [
    [0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2],
    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    # Add more weight combinations to test sensitivity
]

# Apply weights and calculate composite score sensitivity
sensitivity_results = {}
for weights in weights_list:
    composite_score = sum(w * result_df[target] for w, target in zip(weights, profit_targets))
    result_df['composite_score'] = composite_score
    top_ranked = result_df.nlargest(10, 'composite_score')
    sensitivity_results[tuple(weights)] = top_ranked

print("Sensitivity analysis of composite scores with different weights:")
for weights, top_df in sensitivity_results.items():
    print(f"Weights: {weights}")
    print(top_df[['contract_symbol', 'composite_score']].head())

Sensitivity analysis of composite scores with different weights:
Weights: (0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2)
      contract_symbol  composite_score
1  SPY240118C00472000              7.3
2  SPY240118C00473000              7.3
3  SPY240118C00474000              7.3
4  SPY240118C00475000              7.3
5  SPY240118C00476000              7.3
Weights: (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
      contract_symbol  composite_score
1  SPY240118C00472000              5.5
2  SPY240118C00473000              5.5
3  SPY240118C00474000              5.5
4  SPY240118C00475000              5.5
5  SPY240118C00476000              5.5


## Additional Statistical Analysis
* Effect Size Measurement: Cohen’s d
* Logistic Regression Analysis
* Cluster Analysis for Trade Patterns

### Effect Size Measurement: Cohen’s d

In [9]:
def cohen_d(group1, group2):
    pooled_std = sqrt((std(group1, ddof=1) ** 2 + std(group2, ddof=1) ** 2) / 2)
    return (mean(group1) - mean(group2)) / pooled_std

for target in profit_targets:
    d = cohen_d(before_1pm[target], after_1pm[target])
    print(f"{target}: Cohen's d = {d:.2f}")


5%: Cohen's d = 0.17
10%: Cohen's d = 0.21
15%: Cohen's d = 0.23
20%: Cohen's d = 0.25
25%: Cohen's d = 0.26
30%: Cohen's d = 0.27
35%: Cohen's d = 0.29
40%: Cohen's d = 0.29
45%: Cohen's d = 0.29
50%: Cohen's d = 0.29


### Logistic Regression Analysis

In [10]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

# Define independent variables (X) and dependent variable (y)
X = result_df[['Day of Week', 'Option Type', 'moneyness', 'Strike Distance', 'DTE']]
y = result_df['5%']  # Predicting whether an option hit a 5% profit

# Convert categorical variables to numerical
X = pd.get_dummies(X, drop_first=True)

# Convert all columns in X to numeric, coercing any non-numeric values to NaN
X = pd.DataFrame(X).apply(pd.to_numeric, errors='coerce').astype(np.float64)

# Check again for rows with NaN or inf in X after coercion
print("\nRows with NaN in X after conversion:")
print(X[X.isna().any(axis=1)])

print("\nRows with inf in X after conversion:")
print(X[np.isinf(X).any(axis=1)])

# Ensure y is numeric and of type float64
y = pd.Series(y).astype(np.float64)

# Drop rows with NaN or inf in X and align y accordingly
mask = ~X.isna().any(axis=1) & ~np.isinf(X).any(axis=1)
X = X[mask].to_numpy()
y = y[mask.index[mask]].to_numpy()  # Align y to X after dropping rows

# Confirm final data types and shapes
print("\nFinal data types in X:")
print(pd.DataFrame(X).dtypes)
print(f"\nShapes - X: {X.shape}, y: {y.shape}")

# Add a constant for intercept
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X).fit()
print("\nLogistic regression results:")
print(model.summary())



Rows with NaN in X after conversion:
Empty DataFrame
Columns: [Strike Distance, DTE, Day of Week_Monday, Day of Week_Thursday, Day of Week_Tuesday, Day of Week_Wednesday, Option Type_Put, moneyness_ITM, moneyness_OTM]
Index: []

Rows with inf in X after conversion:
Empty DataFrame
Columns: [Strike Distance, DTE, Day of Week_Monday, Day of Week_Thursday, Day of Week_Tuesday, Day of Week_Wednesday, Option Type_Put, moneyness_ITM, moneyness_OTM]
Index: []

Final data types in X:
0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
8    float64
dtype: object

Shapes - X: (87601, 9), y: (87601,)
Optimization terminated successfully.
         Current function value: 0.438975
         Iterations 6

Logistic regression results:
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                87601
Model:                          Logit   Df Residuals:       

In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                87601
Model:                          Logit   Df Residuals:                    87591
Method:                           MLE   Df Model:                            9
Date:                Wed, 19 Mar 2025   Pseudo R-squ.:                 0.01823
Time:                        21:45:38   Log-Likelihood:                -38455.
converged:                       True   LL-Null:                       -39169.
Covariance Type:            nonrobust   LLR p-value:                7.703e-302
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5223      0.287      5.296      0.000       0.959       2.086
x1             0.0316      0.003      9.591      0.000       0.025       0.038
x2             0.0150      0.012      1.238      0.216      -0.009       0.039
x3             0.1101      0.034      3.252      0.001       0.044       0.176
x4             0.2114      0.032      6.538      0.000       0.148       0.275
x5             0.3045      0.034      8.938      0.000       0.238       0.371
x6             0.4284      0.034     12.700      0.000       0.362       0.494
x7            -0.4325      0.019    -22.583      0.000      -0.470      -0.395
x8            -0.1146      0.286     -0.401      0.688      -0.675       0.446
x9             0.4324      0.286      1.512      0.130      -0.128       0.993
==============================================================================
"""

### Cluster Analysis for Trade Patterns

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

# Ensure 'DTE' and 'Strike Distance' are numeric, and convert other columns to dummy variables
features = result_df[['DTE', 'Strike Distance', 'moneyness', 'Option Type']].copy()
features['DTE'] = pd.to_numeric(features['DTE'], errors='coerce')
features['Strike Distance'] = pd.to_numeric(features['Strike Distance'], errors='coerce')
features = pd.get_dummies(features, drop_first=True)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Fit K-means clustering
kmeans = KMeans(n_clusters=4, random_state=0)
result_df['cluster'] = kmeans.fit_predict(features_scaled)

# Confirm that result_df is fully numeric before grouping
result_df = result_df.apply(pd.to_numeric, errors='coerce')

# Summarize cluster characteristics, dropping any columns that might still have NaNs
cluster_summary = result_df.groupby('cluster').mean(numeric_only=True)
print(cluster_summary)


         Date  Day of Week  Time  Before 1 PM  entry_time  contract_symbol  \
cluster                                                                      
0         NaN          NaN   NaN          NaN         NaN              NaN   
1         NaN          NaN   NaN          NaN         NaN              NaN   
2         NaN          NaN   NaN          NaN         NaN              NaN   
3         NaN          NaN   NaN          NaN         NaN              NaN   

         moneyness  Option Type  Strike Distance       DTE  ...       10%  \
cluster                                                     ...             
0              NaN          NaN        -2.833539  0.961494  ...  0.717226   
1              NaN          NaN        -2.844207  0.961340  ...  0.771654   
2              NaN          NaN         1.595954  0.930396  ...  0.668026   
3              NaN          NaN         1.602608  0.930020  ...  0.855157   

              15%       20%       25%       30%       35%       40% 

In [13]:
cluster_summary.transpose()

cluster,0,1,2,3
Date,NaN,NaN,NaN,NaN
Day of Week,NaN,NaN,NaN,NaN
Time,NaN,NaN,NaN,NaN
Before 1 PM,NaN,NaN,NaN,NaN
entry_time,NaN,NaN,NaN,NaN
contract_symbol,NaN,NaN,NaN,NaN
moneyness,NaN,NaN,NaN,NaN
Option Type,NaN,NaN,NaN,NaN
Strike Distance,-2.833539,-2.844207,1.595954,1.602608
DTE,0.961494,0.961340,0.930396,0.930020
